# Estimation of Navier-Stokes equation
## Done by: Andreea-Ioana Florea

In this project I aim at developing a PINN for estimation of the 2D Navier-Stokes equation.

## Imports and dependencies and whatnot

In [30]:
!pip install deepxde

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.12/dist-packages/tensorflow/python/ops/functional_ops.py", line 661, in scan
    _, _, r_a = while_loop.while_loop(  File "/usr/local/lib/python3.12/dist-packages/tensorflow/python/ops/while_loop.py", line 488, in while_loop
    loop_vars = body(*loop_vars)  File "/usr/local/lib/python3.12/dist-packages/tensorflow/python/ops/while_loop.py", line 479, in <lambda>
    body = lambda i, lv: (i + 1, orig_body(*lv))  File "/usr/local/lib/python3.12/dist-packages/tensorflow/python/ops/functional_ops.py", line 653, in compute
    return (next_i, flat_a_out, tas)  File "/usr/local/lib/python3.12/dist-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),
ERROR

In [31]:
import tensorflow as tf
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

In [32]:
rho = 1  # Density
mu = 0.005   # Viscosity
u_in = 1
D = 1
L = 2

channel = dde.geometry.Rectangle(xmin=[-L / 2, -D / 2], xmax=[L / 2, D / 2])
cylinder = dde.geometry.Disk([0, 0], 0.1)

geom_space = channel - cylinder

geom_time = dde.geometry.TimeDomain(0, 5)
geom = dde.geometry.GeometryXTime(geom_space, geom_time)

In [33]:
def pde(x, y):
    # x = [x, y, t], y = [u, v, p]
    u = y[:, 0:1]
    v = y[:, 1:2]
    p = y[:, 2:3]

    u_x = dde.grad.jacobian(y, x, i=0, j=0)
    u_y = dde.grad.jacobian(y, x, i=0, j=1)
    u_t = dde.grad.jacobian(y, x, i=0, j=2)

    v_x = dde.grad.jacobian(y, x, i=1, j=0)
    v_y = dde.grad.jacobian(y, x, i=1, j=1)
    v_t = dde.grad.jacobian(y, x, i=1, j=2)

    p_x = dde.grad.jacobian(y, x, i=2, j=0)
    p_y = dde.grad.jacobian(y, x, i=2, j=1)

    u_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    u_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)

    v_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
    v_yy = dde.grad.hessian(y, x, component=1, i=1, j=1)

    # Continuity
    continuity = u_x + v_y

    # Momentum equations
    mom_u = u_t + u * u_x + v * u_y + p_x / rho - mu * (u_xx + u_yy)
    mom_v = v_t + u * v_x + v * v_y + p_y / rho - mu * (v_xx + v_yy)

    return [continuity, mom_u, mom_v]

def inlet_boundary(x, on_boundary):
    return on_boundary and np.isclose(x[0], -L / 2)

def outlet_boundary(x, on_boundary):
    return on_boundary and np.isclose(x[0], L / 2)

def top_bottom_boundary(x, on_boundary):
    return on_boundary and (np.isclose(x[1], D / 2) or np.isclose(x[1], -D / 2))

def cylinder_boundary(x, on_boundary):
    return on_boundary and (x[0] ** 2 + x[1] ** 2 <= (0.1 + 1e-6) ** 2)


def inlet_u(x):
    y = x[:, 1:2]
    return 4 * u_in * (y + D / 2) * (D / 2 - y) / D ** 2

bc_inlet_u = dde.DirichletBC(geom, inlet_u, inlet_boundary, component=0)
bc_inlet_v = dde.DirichletBC(geom, lambda x: 0.0, inlet_boundary, component=1)

bc_wall_u = dde.DirichletBC(geom, lambda x: 0.0, top_bottom_boundary, component=0)
bc_wall_v = dde.DirichletBC(geom, lambda x: 0.0, top_bottom_boundary, component=1)

bc_cyl_u = dde.DirichletBC(geom, lambda x: 0.0, cylinder_boundary, component=0)
bc_cyl_v = dde.DirichletBC(geom, lambda x: 0.0, cylinder_boundary, component=1)

bc_outlet_p = dde.DirichletBC(geom, lambda x: 0.0, outlet_boundary, component=2)

ic_u = dde.IC(geom, lambda x: 0.0, lambda x, on_initial: on_initial, component=0)
ic_v = dde.IC(geom, lambda x: 0.0, lambda x, on_initial: on_initial, component=1)
ic_p = dde.IC(geom, lambda x: 0.0, lambda x, on_initial: on_initial, component=2)

bcs = [bc_inlet_u, bc_inlet_v, bc_wall_u, bc_wall_v, bc_cyl_u, bc_cyl_v, bc_outlet_p, ic_u, ic_v, ic_p]

In [34]:
data = dde.data.TimePDE(
    geom,
    pde,
    bcs,
    num_domain=5000,
    num_boundary=2500,
    num_initial=1000
)

In [35]:
# Define neural network
net = dde.maps.FNN([3] + [128] * 4 + [3], "tanh", "Glorot uniform")
model = dde.Model(data, net)

In [36]:
# Compile and train
model.compile("adam", lr=1e-3)
losshistory, train_state = model.train(epochs=15000)

model.compile("L-BFGS")
losshistory, train_state = model.train(iterations=500)

Compiling model...
'compile' took 0.008315 s

Training model...

0         [1.14e-01, 2.55e-02, 5.50e-03, 4.59e-01, 1.36e-02, 4.58e-03, 4.23e-02, 1.21e-03, 2.47e-02, 9.35e-03, 8.98e-04, 1.17e-02, 1.08e-02]    [1.14e-01, 2.55e-02, 5.50e-03, 4.59e-01, 1.36e-02, 4.58e-03, 4.23e-02, 1.21e-03, 2.47e-02, 9.35e-03, 8.98e-04, 1.17e-02, 1.08e-02]    []  
1000      [1.07e-02, 4.61e-03, 2.15e-03, 2.38e-02, 1.93e-03, 6.18e-04, 1.83e-02, 9.58e-04, 9.68e-04, 1.36e-04, 6.68e-03, 1.57e-03, 6.54e-04]    [1.07e-02, 4.61e-03, 2.15e-03, 2.38e-02, 1.93e-03, 6.18e-04, 1.83e-02, 9.58e-04, 9.68e-04, 1.36e-04, 6.68e-03, 1.57e-03, 6.54e-04]    []  
2000      [1.48e-02, 4.36e-03, 1.35e-03, 1.35e-02, 5.22e-04, 9.95e-04, 1.30e-02, 1.28e-03, 1.91e-04, 3.70e-05, 5.01e-03, 5.92e-04, 8.50e-04]    [1.48e-02, 4.36e-03, 1.35e-03, 1.35e-02, 5.22e-04, 9.95e-04, 1.30e-02, 1.28e-03, 1.91e-04, 3.70e-05, 5.01e-03, 5.92e-04, 8.50e-04]    []  
3000      [2.70e-02, 6.72e-03, 1.31e-03, 1.06e-02, 2.10e-04, 8.43e-04, 8.03e-03, 9.22e

KeyboardInterrupt: 

In [37]:
# --- Speed magnitude animation (MP4) ---
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle

# Grid for visualization
nx, ny = 200, 100
x = np.linspace(-L/2, L/2, nx)
y = np.linspace(-D/2, D/2, ny)
X, Y = np.meshgrid(x, y)
XY = np.vstack([X.ravel(), Y.ravel()]).T

# Time frames
n_frames = 300
t_vals = np.linspace(0, 5, n_frames)

# Cylinder mask (radius 0.1 at center)
R = 0.1
cyl_mask = (X**2 + Y**2) <= R**2

# Helper to predict speed magnitude at time t
def speed_mag_at_t(t):
    t_col = np.full((XY.shape[0], 1), t)
    XYT = np.hstack([XY, t_col])
    pred = model.predict(XYT)  # [u, v, p]
    u = pred[:, 0].reshape(ny, nx)
    v = pred[:, 1].reshape(ny, nx)
    speed = np.sqrt(u**2 + v**2)
    speed = np.where(cyl_mask, np.nan, speed)  # mask cylinder
    return speed

# Precompute fields and color scale
speeds = [speed_mag_at_t(t) for t in t_vals]
vmin = min(np.nanmin(s) for s in speeds)
vmax = max(np.nanmax(s) for s in speeds)

fig, ax = plt.subplots(figsize=(6, 3))
ax.set_xlabel("x")
ax.set_ylabel("y")
title = ax.set_title(f"Speed magnitude, t={t_vals[0]:.2f}")

# Draw cylinder outline
cyl = Circle((0, 0), R, edgecolor="black", facecolor="white", zorder=10)
ax.add_patch(cyl)

# First frame
cax = ax.contourf(X, Y, speeds[0], levels=50, cmap="jet", vmin=vmin, vmax=vmax)
cb = fig.colorbar(cax, ax=ax)
cb.set_label("Speed magnitude")

def update(frame):
    for coll in list(ax.collections):
        coll.remove()
    ax.contourf(X, Y, speeds[frame], levels=50, cmap="jet", vmin=vmin, vmax=vmax)
    title.set_text(f"Speed magnitude, t={t_vals[frame]:.2f}")
    return None

ani = animation.FuncAnimation(fig, update, frames=n_frames, blit=False)

# Save as MP4 (requires ffmpeg)
ani.save("speed_magnitude.mp4", writer="ffmpeg", dpi=150, fps=30)
plt.close(fig)

print("Saved: speed_magnitude.mp4")

Saved: speed_magnitude.mp4
